In [2]:
!pip install beautifulsoup4
!pip install jq
!pip install -U transformers huggingface_hub
!pip install -q bitsandbytes torch accelerate
!pip install langchain faiss-cpu sentence-transformers langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 8.5 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 60.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 14.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 57.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [43]:
import requests
import warnings
warnings.simplefilter('ignore')
import re
from datetime import date
from langchain import hub
import multiprocessing
from bs4 import BeautifulSoup
from langchain.document_transformers import EmbeddingsRedundantFilter
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import streamlit as st
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings , HuggingFaceBgeEmbeddings
from transformers import AutoModelForCausalLM , AutoTokenizer , BitsAndBytesConfig , pipeline ,VitsModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser , JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_community.llms import HuggingFaceEndpoint
from huggingface_hub import login
from langchain_community.document_loaders import JSONLoader
from langchain.document_transformers import LongContextReorder
from langchain.chains import RetrievalQA
import torch
from getpass import getpass
import os
import json


KeyboardInterrupt



In [ ]:
login()

In [6]:
home_page_url = "https://ticka.world/shop/"
sub_pages_url="https://ticka.world/shop/page/"
home_page = requests.get(home_page_url)
sub_page = requests.get(home_page_url)
soup = BeautifulSoup(home_page.content, "html.parser")

In [7]:
results = soup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")

In [4]:
def preprocessProductName(productName):
    productName= productName.lower().replace('–','').replace('  ',' ').split(' ')
    sep='-'
    res=sep.join(productName)
    return res

In [9]:
def removeWhiteSpaces(ch):
    return " ".join(ch.split())
def getProductsInfo(productName):
     productUrl=f'https://ticka.world/product/{productName}'
    page=requests.get(productUrl)
    if page.status_code !=200 :
        return "",""
    scoup=BeautifulSoup(page.content,"html.parser")
    infoDiv=scoup.find("ul",class_="list-unstyled")
    details=infoDiv.find_all("span",class_="details")
    rate=infoDiv.find("li",class_="clearfix")
    prefix=["Store Name : ","vendor Name : ","Adresse : "]
    store_Name=removeWhiteSpaces(details[0].text)
    rate=removeWhiteSpaces(rate.text)
    return store_Name,rate

In [10]:
def check_stroe_if_exist(data,store_Name):
    store_index=[index for index , name in enumerate(data) if name["Store Name : "]==store_Name ]
    return store_index

In [11]:
def fetchItems(results,data):
    job_elements = results.find_all("div",class_="product-info")
    for i  in tqdm(range(len(job_elements))) :
        title_element = job_elements[i].find("h2", class_="woocommerce-loop-product__title")
        price=job_elements[i].find("span",class_="woocommerce-Price-amount amount")
        price=re.sub('[^0-9.]','',str(price))+" TND"
        title_element=preprocessProductName(title_element.text)
        store_name,rate=getProductsInfo(title_element)
        check_store=check_stroe_if_exist(data,store_name)
        if(len(check_store)==0):
            data.append({"Store Name : ":store_name,"rate : ":rate,"Products : ":[]})
            check_store=[len(data)-1]
        data[check_store[0]]["Products : "].append( {"product":title_element,"price ":price})
    return data

In [12]:
def getItems(data):
    for num_page in range(1,20):
        sub_page = requests.get(sub_pages_url+str(num_page))
        scoup = BeautifulSoup(sub_page.content, "html.parser")
        results = scoup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")
        data=fetchItems(results,data)
    return data

In [13]:
data=getItems([])

100%|██████████| 20/20 [01:14<00:00,  3.70s/it]


In [14]:
def getTickaInformations(tickaInformation):
    landingPageUrl="https://ticka.world/"
    page=requests.get(landingPageUrl)
    scoup = BeautifulSoup(page.content,"html.parser")
    questions=scoup.find_all("a",class_="haru-accordion-title")
    result=scoup.find_all("div",class_="haru-tab-content haru-clearfix")
    activeItem=scoup.find("div",class_="haru-tab-content haru-clearfix active")
    full_text="Question : "+questions[0].text+"\nReponse : "+activeItem.find("span").text
    tickaInformation.append({"content":full_text})
    for i in tqdm(range(len(result))):
        full_text="Question : "+questions[i+1].text+"\nReponse : "+result[i].find("span").text
        tickaInformation.append({"content":full_text})
    return tickaInformation

In [15]:
tickaInformation=getTickaInformations([])

100%|██████████| 11/11 [00:00<00:00, 18008.33it/s]


In [17]:
data.append({"Informations":tickaInformation})
data={"StoreInformations":data}

In [18]:
data=json.dumps(data)

In [19]:
with open("data.json","w") as file :
    file.write(data)

In [5]:
def getTokenizer(model_repo):
    tokenizer=AutoTokenizer.from_pretrained(model_repo)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return tokenizer

def uploadModel(model_repo):
    tokenizer = getTokenizer(model_repo)
#     compute_dtype = getattr(torch, "float16")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
#      bnb_4bit_compute_dtype=compute_dtype,
#      bnb_4bit_quant_type="nf4",
#      bnb_4bit_use_double_quant=False,
    )
    model=AutoModelForCausalLM.from_pretrained(model_repo,
                             quantization_config=bnb_config)
    llm=pipeline(
        model=model,
        task="text-generation",
        tokenizer=tokenizer,
        temperature=0.6,
        return_full_text=False,
        do_sample=True,
        max_new_tokens=512,
    )
    mistral = HuggingFacePipeline(pipeline=llm)
    return mistral

In [15]:
def loadJsonData():
    loader = JSONLoader(
        file_path='/kaggle/input/datajsonnew/data.json',
        jq_schema='.StoreInformations[]',
        text_content=False
    )
    return loader.load()

In [7]:
def split_text():
    json_data = loadJsonData()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200
    )
    doc=text_splitter.split_documents(json_data)
    return doc

In [8]:
def getRetiver(embed):
    doc=split_text()
    db=FAISS.from_documents(
        doc,
        embed
    )
    retiver=db.as_retriever(
        search_type="similarity", search_kwargs={"k": 5, "include_metadata": True}
    )
    return retiver

In [9]:
embed1=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
embed2=HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-dot-v1")

filter_embed= HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [10]:
def merge_retrievers():
    retriever1=getRetiver(embed1)
    retriever2=getRetiver(embed2)
    filter = EmbeddingsRedundantFilter(embeddings=filter_embed) #Delete Redundant docs
    reordering = LongContextReorder() #tackle long context llms problems
    lotr=MergerRetriever(retrievers=[retriever1,retriever2])
    pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
    retriever = ContextualCompressionRetriever(
        base_compressor=pipeline,
        base_retriever=lotr,
        search_kwargs={"k": 5, "include_metadata": True},
    )
    return retriever

In [32]:
def getCompletion(model):
    retiver= merge_retrievers()
    prompt = hub.pull("rlm/rag-prompt-mistral")
    qa_chain = RetrievalQA.from_chain_type(
        model, retriever=retiver, chain_type_kwargs={"prompt": prompt}
    )
#     result=qa_chain({"query":question})
    return qa_chain

# Mistral

In [12]:
model_repo ="mistralai/Mistral-7B-Instruct-v0.2"
model=uploadModel(model_repo)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [27]:
result=getCompletion(model,question="Can you tell me little about this platform ?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
print(result)

 Ticka is an online platform where users can create and purchase customized products with unique designs from local artists. The platform offers a wide range of customizable items, including t-shirts, tote bags, mousepads, stickers, mugs, posters, puzzles, keychains, and more. If you have any questions or need assistance, you can contact Ticka's customer support team through their website or by emailing customer-service@ticka.com.
